In [ ]:
import sys 
#path to append: "./ChimpRec/Code"
sys.path.append(...)

from chimplib.imports import os, np, Image
from chimplib.utils import yolo_to_pixel_coord, face_to_yolo_relative_to_body

In [ ]:
#Path of the test set create for body and face detection evaluation 
test_dataset_path = ...
#Path to image directory of this set
images_dir = f"{test_dataset_path}/images"
#Path to label directory of this set
labels_dir = f"{test_dataset_path}/labels/obj_train_data"
#path of the folder in which the dataset is to be stored
output_dataset = ...

In [ ]:
# @input:
# face: face bbox in pixel format (x1, y1, x2, y2)
# bodies: list of bounding bboxes in YOLO format (x_center, y_center, width, height)
# width: image width in pixels
# height: image height in pixels
# @output:
# bounding box of the corresponding body in pixel format (x1, y1, x2, y2) or None if not found
def get_corresponding_body(face, bodies,  width, height):
    fx1, fy1, fx2, fy2 = face  
    best_body = None
    # We look for the body with the greatest inclusion of the face
    best_overlap = 0 

    for body in bodies:
        bx1, by1, bx2, by2 = yolo_to_pixel_coord(body, width, height)

        #Check that the face is fully contained within the body
        if bx1 <= fx1 and bx2 >= fx2 and by1 <= fy1 and by2 >= fy2:
            #Face and body area calculation 
            face_area = (fx2 - fx1) * (fy2 - fy1)
            body_area = (bx2 - bx1) * (by2 - by1)

            # We maximize the face_area / body_area ratio to choose the body closest in size to the face, and avoid choosing a body 
            # that's too large probably not corresponding to the face 
            overlap = face_area / body_area
            if overlap > best_overlap:
                best_overlap = overlap
                best_body = body 

    if best_body == None: 
        return None
    return yolo_to_pixel_coord(best_body, width, height)


In [ ]:
counter = 0
for root, _, files in os.walk(labels_dir):  # On ignore la liste des sous-dossiers avec "_"
    for file in files:
        faces = []
        bodies = []
        
        with open(f"{labels_dir}/{file}", "r") as current_file:
            for line in current_file:
                values = line.strip().split(" ")

                #1 is the id assigned to body bbox annotations and 0 to face bbox annotations
                if values[0] == "1":
                    bodies.append(tuple(map(float, values[1:])))
                elif values[0] == "0":
                    faces.append(tuple(map(float, values[1:])))

        #open the image corresponding to the current label file
        img = Image.open(images_dir + f"/{file[:-4]}.png")
        frame = np.array(img)
        height, width, _ = frame.shape

        face_coord = None 
        body_coord = None
        for face in faces: 
            #Find the body corresponding to the face currently being treated
            face_coord = tuple(yolo_to_pixel_coord(face, width, height))
            body_coord = get_corresponding_body(face_coord, bodies, width, height)

            if body_coord is None: 
                continue

            fx1, fy1, fx2, fy2 = face_coord
            bx1, by1, bx2, by2 = body_coord

            #crop the image around the body then save in the right place
            cropped_body = frame[by1:by2, bx1:bx2]
            image_pil = Image.fromarray(np.uint8(cropped_body))
            image_pil.save(f"{output_dataset}/images/body{counter}.png")

            #recalculate the face bbox according to the iamge crop around the body and save in the right place
            face_yolo_coord = face_to_yolo_relative_to_body((bx1, by1, bx2, by2), (fx1, fy1, fx2, fy2))
            with open(f"C{output_dataset}/labels/body{counter}.txt", "w") as file:
                file.write("0 " + " ".join(map(str, face_yolo_coord))) 
            counter+= 1
